# IMDB Investigation

### Import Library

In [2]:
# import
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook


### Import Data

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

basics = pd.read_csv(basics_url, low_memory=False, sep='\t')
akas = pd.read_csv(akas_url,  low_memory=False, sep='\t')
ratings = pd.read_csv(ratings_url, low_memory=False, sep='\t')

### Review Dataframes

In [3]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
#creating backup of 'basics' dataframe
df_b_save = basics.copy()

In [5]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
df_a_save = akas.copy()

In [7]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1987
1,tt0000002,5.8,265
2,tt0000003,6.5,1848
3,tt0000004,5.5,178
4,tt0000005,6.2,2630


In [8]:
df_r_save = ratings.copy()

### Preprocessing Cleanup

#### title.basics

##### Find NAN Values

In [9]:
basics = basics.replace({'\\N':np.nan})

##### Evaluating NAN values in columns

In [10]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1346359
endYear           9896995
runtimeMinutes    7037933
genres             451818
dtype: int64

In [11]:
#Display the percentage of missing values by row. 
print(basics.isna().sum()/len(basics)*100)

tconst             0.000000
titleType          0.000000
primaryTitle       0.000110
originalTitle      0.000110
isAdult            0.000010
startYear         13.454284
endYear           98.901540
runtimeMinutes    70.330682
genres             4.515057
dtype: float64


##### Dropping column endYear since this primarily an investigation of movies

In [12]:
basics.drop(['endYear'], axis=1, inplace=True)

##### Eliminate movies that are null for runtimeMinutes

In [13]:
basics.dropna(subset=['runtimeMinutes'], inplace=True)

##### Eliminate movies that are null for genre

In [14]:
basics.dropna(subset=['genres'], inplace=True)

##### keep only titleType==movie

In [15]:
basics = basics[basics['titleType'] == 'movie']

##### Eliminating NAN values for 'startYear'

In [16]:
basics.dropna(subset=['startYear'], inplace=True)

##### NAN values resolved

In [17]:
print(basics.isna().sum()/len(basics)*100)

tconst            0.0
titleType         0.0
primaryTitle      0.0
originalTitle     0.0
isAdult           0.0
startYear         0.0
runtimeMinutes    0.0
genres            0.0
dtype: float64


##### Modifying Datatypes to conform to data dictionary

In [18]:
basics = basics.astype({"tconst":"string","titleType":"string","primaryTitle":"string","originalTitle":"string", "isAdult":"int", "startYear":"int","runtimeMinutes":"int", "genres":"string"})

In [19]:
basics.dtypes

tconst            string
titleType         string
primaryTitle      string
originalTitle     string
isAdult            int32
startYear          int32
runtimeMinutes     int32
genres            string
dtype: object

##### keep startYear 2000-2022

In [20]:
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]

##### Eliminate movies that include "Documentary" in genre

In [21]:
basics = basics[~basics['genres'].str.contains(r'documentary', case=False)]


#### title.akas

##### Find NAN Values

In [22]:
akas =akas.replace({'\\N':np.nan})

##### Filtering out all non US Films

In [23]:
akas = akas[akas['region'] == 'US']

##### Evaluating NAN values in columns

In [24]:
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1447458
types               470217
attributes         1404479
isOriginalTitle       1342
dtype: int64

In [25]:
print(akas.isna().sum()/len(akas)*100)

titleId             0.000000
ordering            0.000000
title               0.000000
region              0.000000
language           99.723934
types              32.396027
attributes         96.762857
isOriginalTitle     0.092458
dtype: float64


##### Dropping Columns Language and Attributes due to lack of data, as well as isOriginalTitle since it doesnt directly relate to investigation

In [26]:
akas.drop(['language','attributes','isOriginalTitle'], axis=1, inplace=True)

##### Replacing NAN values with Unknown for types column

In [27]:
type_of = 'unknown'
akas['types'].fillna(value = type_of, inplace = True)

##### NAN Values Resolved

In [28]:
akas.isna().sum()

titleId     0
ordering    0
title       0
region      0
types       0
dtype: int64

##### Modifying Datatypes to conform to Data Dictionary

In [29]:
akas = akas.astype({"titleId":"string","title":"string","region":"string","types":"string"})

In [30]:
akas.dtypes

titleId     string
ordering     int64
title       string
region      string
types       string
dtype: object

#### title.ratings

##### Find NAN Values

In [31]:
ratings = ratings.replace({'\\N':np.nan})

In [32]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

* No NAN values found

##### Modifying Datatypes to conform to data dictionary

In [33]:
ratings = ratings.astype({"tconst":"string"})

In [34]:
ratings.dtypes

tconst            string
averageRating    float64
numVotes           int64
dtype: object

#### Filtering for Title Basics with akas dataframe for US only

In [35]:
keepers1 =basics['tconst'].isin(akas['titleId'])
keepers1

34802        True
61114        True
67666        True
86793        True
93930        True
            ...  
10006599     True
10006638    False
10006683     True
10006767    False
10006857    False
Name: tconst, Length: 147963, dtype: bool

In [36]:
basics = basics[keepers1]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama
...,...,...,...,...,...,...,...,...
10006055,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,74,Drama
10006450,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy"
10006590,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
10006599,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"


#### Filtering for ratings with akas dataframe for US only

In [37]:
keepers2 =ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1330208    False
1330209    False
1330210    False
1330211    False
1330212    False
Name: tconst, Length: 1330213, dtype: bool

In [38]:
ratings = ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1987
1,tt0000002,5.8,265
4,tt0000005,6.2,2630
5,tt0000006,5.1,182
6,tt0000007,5.4,824
...,...,...,...
1330174,tt9916200,8.1,231
1330175,tt9916204,8.2,264
1330182,tt9916348,8.3,18
1330183,tt9916362,6.4,5418


### Saving modified dataframes to file for later use

In [39]:
basics.to_csv("Data/title_basics1.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas1.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings1.csv.gz",compression='gzip',index=False)

## Project 3 - Part 2 Extract from TMDB (Core)

### Loading Files

In [40]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas1.csv.gz',
 'title_basics1.csv.gz',
 'title_ratings1.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [41]:
# Load API credentials from JSON file
#with open('/Users/Paul Lipska/.secret/tmdb_api.json', 'r') as f:
with open('/Users/paull/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)

# Set API key
tmdb.API_KEY = login['api-key']

# Create a movie object using the Movies function from tmdb
movie = tmdb.Movies(603)

# Retrieve movie information
info = movie.info()

### Loading df from previous exercise

In [42]:
# Load in the dataframe from project part 1
basics = pd.read_csv('Data/title_basics1.csv.gz')

In [43]:
YEARS_TO_GET = [2000,2001]

In [44]:
errors = []

In [45]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama


In [46]:
#get movie certification
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movei object fro the current id
    movie = tmdb.Movies(movie_id)
    
    #save the .info .release dictionaries
    info = movie.info()
    
    releases = movie.releases()
    #Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1'] == 'US':
            ## save a 'certification' key in info with the certification
            info['certification'] = c['certification']
            
    return info

In [47]:
#append function
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [48]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    #if it does not exist: create it
    if file_exists == False:
    #save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)
        
    #saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    
    #saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    movie_ids

    #Load existing data from json into a dataframe called 'previous_df'
    previous_df = pd.read_json(JSON_FILE)

    previous_df

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
        

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])



    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/208 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/233 [00:00<?, ?it/s]

## Project 3 - Part 3 Construct MySql db from data (Core)

#### Examining saved dataframes from previous exercise

In [3]:
#loading title basics
basics = pd.read_csv('Data/title_basics1.csv.gz')

In [4]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama
...,...,...,...,...,...,...,...,...
86945,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,74,Drama
86946,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy"
86947,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama
86948,tt9916190,movie,Safeguard,Safeguard,0,2020,95,"Action,Adventure,Thriller"


In [5]:
#copying dataframe for editing
df_basics = basics.copy()

#### Removing columns that are not needed for MYSQL db

In [6]:
# remove titleType, originalTitle, isAdult
df_basics.drop(columns=['titleType', 'originalTitle', 'isAdult'], inplace=True)

In [7]:
df_basics.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres
0,tt0035423,Kate & Leopold,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,70,Drama
2,tt0069049,The Other Side of the Wind,2018,122,Drama
3,tt0088751,The Naked Monster,2005,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,Crime and Punishment,2002,126,Drama


In [8]:
df_basics['genres'] = pd.DataFrame(df_basics['genres'].str.split(','))
                            

In [9]:
df_basics

,tconst,primaryTitle,startYear,runtimeMinutes,genres
0,tt0035423,Kate & Leopold,2001,118,"[Comedy, Fantasy, Romance]"
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,70,[Drama]
2,tt0069049,The Other Side of the Wind,2018,122,[Drama]
3,tt0088751,The Naked Monster,2005,100,"[Comedy, Horror, Sci-Fi]"
4,tt0096056,Crime and Punishment,2002,126,[Drama]
...,...,...,...,...,...
86945,tt9914942,Life Without Sara Amat,2019,74,[Drama]
86946,tt9915872,The Last White Witch,2019,97,"[Comedy, Drama, Fantasy]"
86947,tt9916170,The Rehearsal,2019,51,[Drama]
86948,tt9916190,Safeguard,2020,95,"[Action, Adventure, Thriller]"


In [10]:
df_basics = df_basics.explode('genres')
df_basics

,tconst,primaryTitle,startYear,runtimeMinutes,genres
0,tt0035423,Kate & Leopold,2001,118,Comedy
0,tt0035423,Kate & Leopold,2001,118,Fantasy
0,tt0035423,Kate & Leopold,2001,118,Romance
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,70,Drama
2,tt0069049,The Other Side of the Wind,2018,122,Drama
...,...,...,...,...,...
86948,tt9916190,Safeguard,2020,95,Action
86948,tt9916190,Safeguard,2020,95,Adventure
86948,tt9916190,Safeguard,2020,95,Thriller
86949,tt9916362,Coven,2020,92,Drama


In [11]:
unique_genres = sorted(df_basics['genres'].unique())
unique_genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

In [12]:
int_ids = range(len(unique_genres))

In [13]:
int_ids

range(0, 25)

In [14]:
id_map = dict(zip(unique_genres,int_ids))
id_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Sport': 20,
 'Talk-Show': 21,
 'Thriller': 22,
 'War': 23,
 'Western': 24}

In [15]:
df_basics['genres'] = df_basics['genres'].map(id_map)

In [16]:
df_basics

,tconst,primaryTitle,startYear,runtimeMinutes,genres
0,tt0035423,Kate & Leopold,2001,118,5
0,tt0035423,Kate & Leopold,2001,118,9
0,tt0035423,Kate & Leopold,2001,118,18
1,tt0062336,The Tango of the Widower and Its Distorting Mi...,2020,70,7
2,tt0069049,The Other Side of the Wind,2018,122,7
...,...,...,...,...,...
86948,tt9916190,Safeguard,2020,95,0
86948,tt9916190,Safeguard,2020,95,2
86948,tt9916190,Safeguard,2020,95,22
86949,tt9916362,Coven,2020,92,7


In [17]:
# Using pd.DataFrame and a dictionary
id_lookup = pd.DataFrame({'genre_name': id_map.keys(),
                         'genre_id':id_map.values()})

In [18]:
id_lookup

,genre_name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4
5,Comedy,5
6,Crime,6
7,Drama,7
8,Family,8
9,Fantasy,9


#### Creating new tables title_genres and genres

In [ ]:
genres['id'] = df['id'].replace(id_map)
df.head(3)

In [62]:
#genre_id
#genre_name

In [65]:
#unique_genres

,genre_name
0,Comedy
1,Fantasy
2,Romance
3,Drama
4,Horror
5,Sci-Fi
6,Biography
7,Mystery
8,Musical
9,Action


In [50]:
#loading title ratings
ratings = pd.read_csv('Data/title_ratings1.csv.gz')

In [57]:
ratings.head()


,tconst,averageRating,numVotes
0,tt0000001,5.7,1987
1,tt0000002,5.8,265
2,tt0000005,6.2,2630
3,tt0000006,5.1,182
4,tt0000007,5.4,824
